In [3]:
import pandas as pd
import re

In [4]:
with open('./data/message.txt', 'r') as f:
    log_data = f.read()

In [49]:
print(log_data)

type=SYSCALL msg=audit(1769623543.594:18582): arch=c000003e syscall=59 success=yes exit=0 a0=59b175294d20 a1=59b1751d3340 a2=59b1752d2f90 a3=59b1751d3340 items=2 ppid=18414 pid=19059 auid=1000 uid=1000 gid=1000 euid=0 suid=0 fsuid=0 egid=1000 sgid=1000 fsgid=1000 tty=pts0 ses=65 comm="sudo" exe="/usr/bin/sudo" subj=unconfined key="exec_log"ARCH=x86_64 SYSCALL=execve AUID="user" UID="user" GID="user" EUID="root" SUID="root" FSUID="root" EGID="user" SGID="user" FSGID="user"
type=EXECVE msg=audit(1769623543.594:18582): argc=4 a0="sudo" a1="tail" a2="-f" a3="/var/log/audit/audit.log"
type=CWD msg=audit(1769623543.594:18582): cwd="/home/user"
type=PATH msg=audit(1769623543.594:18582): item=0 name="/usr/bin/sudo" inode=1124 dev=fc:00 mode=0104755 ouid=0 ogid=0 rdev=00:00 nametype=NORMAL cap_fp=0 cap_fi=0 cap_fe=0 cap_fver=0 cap_frootid=0OUID="root" OGID="root"
type=PATH msg=audit(1769623543.594:18582): item=1 name="/lib64/ld-linux-x86-64.so.2" inode=39819 dev=fc:00 mode=0100755 ouid=0 ogid=0

In [5]:
from typing import List, Dict

def normalize_audit_log(data: str) -> List[Dict]:
    lines = data.strip().split('\n')
    events = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # 1. แยก type และส่วน msg=audit(...)
        header_match = re.match(r'type=([A-Z_]+) msg=audit\([^)]+\): (.*)', line)
        if not header_match:
            continue

        event_type = header_match.group(1)
        rest = header_match.group(2)

        # 2. Dictionary สำหรับเก็บ key-value ทั้งหมด
        pairs: Dict[str, str] = {}

        # 3. จับ key=value ที่อยู่นอก msg='' ก่อน (เช่น pid, uid, auid, ...)
        for match in re.finditer(r'(\w+)=([^ ]+)(?=\s|$)', rest):
            key, value = match.groups()
            # ลบเครื่องหมาย " หรือ ' ที่อาจหลงเหลือ
            value = value.strip('"').strip("'")
            pairs[key] = value

        # 4. จับส่วน msg='...' หรือ msg="..." แล้วแยก key=value ภายใน
        msg_match = re.search(r"msg=['\"](.*?)['\"](?=\s|$|\w+=)", rest, re.DOTALL)
        if msg_match:
            msg_content = msg_match.group(1)
            # แยก key=value ภายใน msg (รองรับทั้ง quoted และ unquoted)
            i = 0
            while i < len(msg_content):
                # ข้าม whitespace
                while i < len(msg_content) and msg_content[i].isspace():
                    i += 1
                if i >= len(msg_content):
                    break

                # หา key=
                eq_pos = msg_content.find('=', i)
                if eq_pos == -1:
                    break
                key = msg_content[i:eq_pos].strip()
                i = eq_pos + 1

                # ดูว่าค่าเริ่มด้วย quote หรือไม่
                if i < len(msg_content) and msg_content[i] in ("'", '"'):
                    quote = msg_content[i]
                    i += 1
                    end_quote = msg_content.find(quote, i)
                    if end_quote == -1:
                        end_quote = len(msg_content)
                    value = msg_content[i:end_quote]
                    i = end_quote + 1
                else:
                    # ไม่มี quote → จนถึง space หรือจบ
                    end = msg_content.find(' ', i)
                    if end == -1:
                        end = len(msg_content)
                    value = msg_content[i:end]
                    i = end

                pairs[key] = value.strip()

        # 5. สร้าง event ที่ standardize แล้ว
        event = {
            "event_type": event_type,
            "pid": pairs.get("pid"),
            "uid": pairs.get("uid"),
            "auid": pairs.get("auid"),
            "ses": pairs.get("ses"),
            "exe": pairs.get("exe"),
            "acct": pairs.get("acct"),
            "op": pairs.get("op"),
            "unit": pairs.get("unit"),
            "comm": pairs.get("comm"),
            "cmd": pairs.get("cmd"),               # จะได้ hex หรือ plain text
            "cwd": pairs.get("cwd"),
            "terminal": pairs.get("terminal"),
            "hostname": pairs.get("hostname"),
            "addr": pairs.get("addr"),
            "res": pairs.get("res"),
            "success": pairs.get("success"),       # สำหรับ SYSCALL
            "proctitle": pairs.get("proctitle"),   # hex หรือ decoded
        }

        # เพิ่ม field เฉพาะบาง type ถ้าต้องการ
        if event_type in ("SYSCALL", "LOGIN"):
            event["syscall"] = pairs.get("syscall")
            event["arch"] = pairs.get("arch")

        events.append(event)

    return events

In [6]:
events = normalize_audit_log(log_data)
df = pd.DataFrame(events)

In [41]:
df

,event_type,uid,auid,exe,terminal,res,op,acct,pid,ses,...,hostname,addr,success,proctitle,syscall,arch,Type_encoded,Command_encoded,Anomaly,anomaly_score
0,SYSCALL,1000,1000,/usr/bin/sudo,NaN,NaN,NaN,NaN,19059,65,...,NaN,NaN,yes,NaN,59,c000003e,5,1,1,1
1,EXECVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1,1
2,CWD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1
3,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1
4,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,EXECVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1,1
1128,CWD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1
1129,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1
1130,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1


In [8]:
useful_cols = [
    'event_type', 'uid', 'auid', 'exe', 'terminal', 'res', 'op', 'acct'
]
df = df[useful_cols + [c for c in df.columns if c not in useful_cols]]

In [42]:
df

,event_type,uid,auid,exe,terminal,res,op,acct,pid,ses,...,hostname,addr,success,proctitle,syscall,arch,Type_encoded,Command_encoded,Anomaly,anomaly_score
0,SYSCALL,1000,1000,/usr/bin/sudo,NaN,NaN,NaN,NaN,19059,65,...,NaN,NaN,yes,NaN,59,c000003e,5,1,1,1
1,EXECVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1,1
2,CWD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1
3,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1
4,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,EXECVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1,1
1128,CWD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1
1129,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1
1130,PATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,1


In [10]:
# ภาพรวม
print(df['event_type'].value_counts())

event_type
PATH          376
SYSCALL       188
EXECVE        188
CWD           188
PROCTITLE     188
USER_ACCT       1
USER_CMD        1
CRED_REFR       1
USER_START      1
Name: count, dtype: int64


In [11]:
# sudo
sudo_events = df[df['exe'].str.contains('sudo', na=False)]
print(sudo_events[['uid', 'auid', 'res']])

    uid  auid                res
0  1000  1000                NaN
6  1000  1000  success'UID="user
7  1000  1000  success'UID="user
8  1000  1000  success'UID="user
9  1000  1000  success'UID="user


In [12]:
# คำสั่งที่รันบ่อย
cmd_count = df['cmd'].value_counts().head(10)
print("คำสั่งที่รันบ่อย:\n", cmd_count)

คำสั่งที่รันบ่อย:
 cmd
7461696C202D66202F7661722F6C6F672F61756469742F61756469742E6C6F67    1
Name: count, dtype: int64


In [13]:
# Event ที่ล้มเหลว
failed = df[df['res'].str.contains('fail|no|error', na=False, case=False)]
print("เหตุการณ์ล้มเหลว:\n", failed[['event_type', 'exe', 'cmd']])

เหตุการณ์ล้มเหลว:
 Empty DataFrame
Columns: [event_type, exe, cmd]
Index: []


In [14]:
# User ธรรมดาเปลี่ยนไป root
root_acct = df[df['acct'] == 'root']
print("พยายามเปลี่ยนเป็น root:\n", root_acct[['uid', 'auid', 'exe', 'cmd']])

พยายามเปลี่ยนเป็น root:
     uid  auid            exe  cmd
8  1000  1000  /usr/bin/sudo  NaN
9  1000  1000  /usr/bin/sudo  NaN


### Isolation Forest

ทำโมเดล Isolation เพื่อหาความผิดปกติ

In [15]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder

In [16]:
le = LabelEncoder()

In [17]:
df['Type_encoded'] = le.fit_transform(df['event_type'])
df['Command_encoded'] = le.fit_transform(df['cmd'].fillna('unknown'))

In [18]:
features = df[['Type_encoded', 'Command_encoded']].values

In [19]:
model = IsolationForest(contamination=0.1, random_state=42)

In [20]:
model.fit(features)

,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",100
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",0.1
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0
,"bootstrap bootstrap: bool, default=FalseIf True, individual trees are fit on random subsets of the trainingdata sampled with replacement. If False, sampling without replacementis performed.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for :meth:`fit`. ``None`` means 1unless in a :obj:`joblib.parallel_backend` context. ``-1`` means usingall processors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo-randomness of the selection of the featureand split values for each branching step and each tree in the forest.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42
,"verbose verbose: int, default=0Controls the verbosity of the tree building process.",0
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fit a wholenew forest. See :term:`the Glossary `... versionadded:: 0.21",False


In [21]:
df['Anomaly'] = model.predict(features)

In [22]:
anomalies_detected = df[df['Anomaly'] == -1]

In [43]:
anomalies_detected

,event_type,uid,auid,exe,terminal,res,op,acct,pid,ses,...,cwd,hostname,addr,success,proctitle,syscall,arch,Type_encoded,Command_encoded,Anomaly
6,USER_ACCT,1000,1000,/usr/bin/sudo,/dev/pts/0,"success'UID=""user",PAM:accounting,user,19059,65,...,NaN,?,?,NaN,NaN,NaN,NaN,6,1,-1
7,USER_CMD,1000,1000,/usr/bin/sudo,pts/0,"success'UID=""user",NaN,NaN,19059,65,...,/home/user,NaN,NaN,NaN,NaN,NaN,NaN,7,0,-1
8,CRED_REFR,1000,1000,/usr/bin/sudo,/dev/pts/0,"success'UID=""user",PAM:setcred,root,19059,65,...,NaN,?,?,NaN,NaN,NaN,NaN,0,1,-1
9,USER_START,1000,1000,/usr/bin/sudo,/dev/pts/0,"success'UID=""user",PAM:session_open,root,19059,65,...,NaN,?,?,NaN,NaN,NaN,NaN,8,1,-1


In [25]:
df['anomaly_score'] = model.fit_predict(features)

In [47]:
df_anomaly_detect = df[df['anomaly_score'] == -1]
df_anomaly_detect

,event_type,uid,auid,exe,terminal,res,op,acct,pid,ses,...,hostname,addr,success,proctitle,syscall,arch,Type_encoded,Command_encoded,Anomaly,anomaly_score
6,USER_ACCT,1000,1000,/usr/bin/sudo,/dev/pts/0,"success'UID=""user",PAM:accounting,user,19059,65,...,?,?,NaN,NaN,NaN,NaN,6,1,-1,-1
7,USER_CMD,1000,1000,/usr/bin/sudo,pts/0,"success'UID=""user",NaN,NaN,19059,65,...,NaN,NaN,NaN,NaN,NaN,NaN,7,0,-1,-1
8,CRED_REFR,1000,1000,/usr/bin/sudo,/dev/pts/0,"success'UID=""user",PAM:setcred,root,19059,65,...,?,?,NaN,NaN,NaN,NaN,0,1,-1,-1
9,USER_START,1000,1000,/usr/bin/sudo,/dev/pts/0,"success'UID=""user",PAM:session_open,root,19059,65,...,?,?,NaN,NaN,NaN,NaN,8,1,-1,-1


### AI Agent #1
ใส่ log ทีละตัว

In [37]:
anomaly_row = df[df['anomaly_score'] == -1].iloc[3]
anomaly_row

event_type                USER_START
uid                             1000
auid                            1000
exe                    /usr/bin/sudo
terminal                  /dev/pts/0
res                success'UID="user
op                  PAM:session_open
acct                            root
pid                            19059
ses                               65
unit                            None
comm                             NaN
cmd                              NaN
cwd                              NaN
hostname                           ?
addr                               ?
success                          NaN
proctitle                        NaN
syscall                          NaN
arch                             NaN
Type_encoded                       8
Command_encoded                    1
Anomaly                           -1
anomaly_score                     -1
Name: 9, dtype: object

In [31]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(
    model="thewindmom/llama3-med42-8b",
    temperature=0.8,
    # base_url="http://localhost:11434"  # default ไม่ต้องใส่
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "คุณคือผู้ดูแลระบบ log และให้คำแนะนำทาง cyber policy ที่เหมาะสม"),
        ("human", "จาก audit log anomaly นี้: \nEvent: {event_type} \nCommand: {cmd}\nEXE: {exe}\nTerminal: {terminal}\nแนะนำ policy cybersecurity ที่เหมาะสม (เช่น SELinux rule, firewall, user permission, alert rule)")
    ]
)

chain = prompt | llm

# ใช้กับ anomaly ที่ detect ได้
anomaly_row = df[df['anomaly_score'] == -1].iloc[0]
result = chain.invoke({
    "event_type": anomaly_row['event_type'],
    "cmd": anomaly_row['cmd'],
    "exe": anomaly_row['exe'],
    "terminal": anomaly_row['terminal']
})

print(result.content)

Based on the provided audit log anomaly:

Event: USER_ACCT
Command: nan
EXE: /usr/bin/sudo
Terminal: /dev/pts/0

Here is a suitable cybersecurity policy recommendation:

1. Implement SELinux rules to restrict access and enforce least privilege for sudo commands:
   - Create custom SELinux contexts for the sudo binary and its associated files.
   - Define new boolean settings for allowing or denying specific actions (e.g., allow_sudo NanCommand).
   - Enforce strict enforcement of these SELinux rules.

2. Configure firewall rules to monitor and control incoming and outgoing traffic related to sudo commands:
   - Set up a network intrusion detection system (NIDS) or an intrusion prevention system (IPS) to alert on suspicious activity.
   - Implement firewall rules blocking incoming connections to the sudo binary from unknown hosts or networks.

3. Establish user permissions policies for sudo usage:
   - Limit sudo access only to necessary users with specific roles.
   - Implement mandato

In [39]:
# หา anomaly ทั้งหมด
anomalies = df[df['anomaly_score'] == -1]

print(f"พบ anomaly ทั้งหมด: {len(anomalies)} รายการ\n")

if not anomalies.empty:
    print("รายการ anomaly ที่ตรวจพบ:")
    print(anomalies[['event_type', 'cmd', 'exe', 'terminal', 'res', 'anomaly_score']])
    print("\n" + "="*80 + "\n")

พบ anomaly ทั้งหมด: 4 รายการ

รายการ anomaly ที่ตรวจพบ:
   event_type                                                cmd  \
6   USER_ACCT                                                NaN   
7    USER_CMD  7461696C202D66202F7661722F6C6F672F61756469742F...   
8   CRED_REFR                                                NaN   
9  USER_START                                                NaN   

             exe    terminal                res  anomaly_score  
6  /usr/bin/sudo  /dev/pts/0  success'UID="user             -1  
7  /usr/bin/sudo       pts/0  success'UID="user             -1  
8  /usr/bin/sudo  /dev/pts/0  success'UID="user             -1  
9  /usr/bin/sudo  /dev/pts/0  success'UID="user             -1  




### Anomaly Agent #2
ใส่ anomaly ทั้งหมดทีละ row

In [40]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(
    model="thewindmom/llama3-med42-8b",
    temperature=0.8,
    # base_url="http://localhost:11434"  # default ไม่ต้องใส่
)

prompt = PromptTemplate.from_template("""
จาก audit log event นี้ที่ถูกตรวจพบว่าเป็น anomaly:
Event Type     : {event_type}
EXE            : {exe}
Command (decoded): {cmd}
Terminal       : {terminal}
Result         : {res}
User/Acct      : {user}

โปรดวิเคราะห์ว่าเหตุการณ์นี้อาจเป็นความผิดปกติแบบใด
และแนะนำแนวทางปรับใช้ policy หรือการป้องกันที่เหมาะสม (เช่น audit rule, sudoers, SELinux, firewall, การจำกัดสิทธิ์ ฯลฯ)
ให้ชัดเจน สั้นกระชับ และปฏิบัติได้จริง
""")

chain = prompt | llm | StrOutputParser()

# วนลูปทุก anomaly
for idx, row in anomalies.iterrows():
    print(f"\nAnomaly #{idx+1} (index {row.name})")
    print("-"*60)
    
    response = chain.invoke({
        "event_type": row.get('event_type', 'N/A'),
        "exe": row.get('exe', 'N/A'),
        "cmd": row.get('cmd', row.get('cmd', 'N/A')),
        "terminal": row.get('terminal', 'N/A'),
        "res": row.get('res', 'N/A'),
        "user": row.get('user', 'N/A')
    })
    
    print(response)
    print("="*80)


Anomaly #7 (index 6)
------------------------------------------------------------
This event is likely an anomaly due to the unusual command "nan". Since it's decoded as an empty string (""), it could potentially indicate a malicious attempt to execute a command with sudo without specifying any actual action. 

To address this, consider implementing the following measures:

1. Audit rule: Configure auditd to monitor and log all sudo activities, ensuring that every command executed by sudo is recorded.
2. Sudoers policy: Enforce strict rules in /etc/sudoers to specify allowed commands for each user or group. Forbid * wildcard usage and ensure explicit command paths are defined.
3. SELinux: Implement a targeted policy to restrict sudo access based on context, allowing only necessary actions while blocking potentially harmful activities.
4. Firewall: Configure the firewall to limit incoming/outgoing traffic from suspicious sources or ports related to SSH or other potential attack vectors